<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Settings

<div class="alert alert-success" role="info">
    <p><b>Note:</b> this data product framework is developed by Naas open source community. You can <a href="https://github.com/sponsors/jupyter-naas" target="_blank">sponsor us</a> if you find it usefull.
</p>
</div>

This notebook allows you to configure and run your data product on your Naas server. *Create an account on [Naas.ai](https://www.naas.ai/free-forever) if you don't already have one.*

Please follow the step below to setup this data product.

## Input

### Install requirements
Install the packages required to run the product.

In [1]:
# !pip install --user --upgrade -r "requirements.txt"

### Import libraries

In [2]:
import naas_data_product
import naas
from naas_drivers import gsheet
import glob
import os
from IPython.display import clear_output
from tqdm import tqdm

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables
**Pre-requisite**
- Duplicate our Google Sheets spreadsheet model
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

**Inputs**
- `abi_spreadsheet`: Google Sheets spreadsheet URL
- `sheet_entity`: Entity sheet name that stores all your personal data

**Outputs**
- `output_dir_plugin`: Cookie used to authenticate Members and API clients.

In [3]:
# Inputs
abi_spreadsheet = naas.secret.get("ABI_SPREADSHEET")
if abi_spreadsheet is None:
    abi_spreadsheet = input("Enter your Google Sheets URL:")
    naas.secret.add("ABI_SPREADSHEET", abi_spreadsheet)
sheet_entity = "ENTITY"

# Outputs
output_dir = os.path.join("outputs", "entity")
output_dir_plugin = os.path.join("outputs", "plugins")

## Model

### Get data from Google Sheet spreadsheet

In [4]:
df_gsheet = gsheet.connect(abi_spreadsheet).get(sheet_name=sheet_entity).fillna("NA")
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

Rows: 1


,ENTITY,LINKEDIN_URL,PROMPT_ICP,PROMPT_SALES_MESSAGINGS,EMAILS,LINKEDIN_LI_AT,LINKEDIN_JSESSIONID,OPENAI_API_KEY
0,Jérémy Ravenel,https://www.linkedin.com/in/jeremyravenel/,"I have 2 ideal customer profile, one is a 'dat...",Create 3 messaging options to engage conversat...,"jeremy@naas.ai, florent@naas.ai",NA,NA,NA


### Store LinkedIn URL
The LinkedIn URL will be used in content and growth engine.

In [5]:
linkedin_url = df_gsheet.loc[0, "LINKEDIN_URL"]
print("LinkedIn URL:", linkedin_url)
pdump(output_dir, linkedin_url, "linkedin_url")

LinkedIn URL: https://www.linkedin.com/in/jeremyravenel/


### Store Emails
The emails will be used for the notifications

In [6]:
emails = df_gsheet.loc[0, "EMAILS"]
print("Emails:", emails)
pdump(output_dir, emails, "emails")

Emails: jeremy@naas.ai, florent@naas.ai


### Store Prompt ICP
The prompt will be used in the growth engine.

In [7]:
prompt_icp = df_gsheet.loc[0, "PROMPT_ICP"]
print("Prompt ICP:", prompt_icp)
pdump(output_dir, prompt_icp, "prompt_icp")

Prompt ICP: I have 2 ideal customer profile, one is a 'data producer' with basic knowledge of Python that could use our Notebook templates to create plugins. 
These plugions are then distributed data via our NaasAI Chat interface.
The other one is a 'data consummer' that will enjoy using NaasAI Chat for its basic LLMs integration but also interested in having its own data available, hence work with the data producer. 
I will give you the occupation from a profile I get from LinkedIn, you will return stricly and only one of the following values inside the simple quotes based on the best match 'DataProducer', 'DataConsummer', 'NotICP' or 'NA' if you don't find a plausible match with the first 3 values.
Don't put the results into quotes.


### Store Sales Messagings
The prompt will be used in the sales engine to create sales messagins options on new sales qualified leads.

In [8]:
prompt_sales_messagings = df_gsheet.loc[0, "PROMPT_SALES_MESSAGINGS"]
print("Prompt Sales Messagings:", prompt_sales_messagings)
pdump(output_dir, prompt_sales_messagings, "prompt_sales_messagings")

Prompt Sales Messagings: Create 3 messaging options to engage conversation with a person on LinkedIn, be brief, concise, casual but professional. Speak as the author of the post Jéméry Ravenel


### Add secrets to Naas

In [9]:
for x in ["LINKEDIN_LI_AT", "LINKEDIN_JSESSIONID", "OPENAI_API_KEY"]:
    value = df_gsheet.loc[0, x]
    if value != "NA":
        print(f"Secret '{x}' to be added:")
        naas.secret.add(x, value)

## Output

### Run plugins

In [10]:
# List plugin files
files = glob.glob("*/**/__plugin__.ipynb", recursive=True)

# Execute notebook
for file in tqdm(files):
    print(f"File: {file}")
    print()
    %run $file
    clear_output()

100%|██████████| 7/7 [01:05<00:00,  9.33s/it]


### Get plugins URL

In [11]:
data = []
plugins = glob.glob(f"{output_dir_plugin}/*.json")

for plugin in plugins:
    plugin_url = naas.asset.find(plugin)
    data.append(
        {
            "name": plugin.split("/")[-1].replace(".json", ""),
            "image": "",
            "url": plugin_url,
            "new_chat_url": f"https://naas.ai/chat/use?plugin_url={plugin_url}"
        }
    )
    
data

[{'name': '⚡️_sales_assitant',
  'image': '',
  'url': 'https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/f88b68b1b2d22ffa623a2c9a6a1dd000ab04ed19f889a0bcdd81bf290874',
  'new_chat_url': 'https://naas.ai/chat/use?plugin_url=https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/f88b68b1b2d22ffa623a2c9a6a1dd000ab04ed19f889a0bcdd81bf290874'},
 {'name': '🚀_growth_assistant',
  'image': '',
  'url': 'https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/e65caf57ac5c44002f2c6b68d4ed15d240daa9367df5eee06aee1792b6a8',
  'new_chat_url': 'https://naas.ai/chat/use?plugin_url=https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/e65caf57ac5c44002f2c6b68d4ed15d240daa9367df5eee06aee1792b6a8'},
 {'name': '🤖_abi',
  'image': '',
  'url': 'https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/2e001709f5f03dfea0f169f121781f136fd82e1c3128a3aba2338ff0fb77',
  'new_chat_url': 'https://naas.ai/chat/use?plugin_url=https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/2e001709f5f03